# Imports

we want to import openCV, mediapipe, numpy

In [ ]:
import cv2
import mediapipe as mp
import numpy

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
  
def draw_styled_landmarks(image, results):

    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 


In [ ]:
cap = cv2.VideoCapture('./videos/Cross.MOV')
frames = []
frameCount = 0
jumpFrame = 21

# Set mediapipe model 

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        if jumpFrame >0: jumpFrame -=1


        print(jumpFrame)

        if jumpFrame == 0:
            if cv2.waitKey(0) & 0xFF == ord('p'):
                frameCount-=1
                cap.set(cv2.CAP_PROP_POS_FRAMES, frameCount)
            else:
                frameCount+=1
        else:
            frameCount+=1

        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        frame = []
        if results.pose_landmarks:
            for res in results.pose_landmarks.landmark:
                if res.visibility > .65:
                    frame.append(res.x)
                    frame.append(res.y)
                    frame.append(res.z)
                else:
                    frame.append(0)
                    frame.append(0)
                    frame.append(0)

            frames.append(frame)
            # Draw landmarks
            draw_styled_landmarks(image, results)

            # Show to screen

        cv2.putText(image, text= str(frameCount), org=(500, 100),
                fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,0,0),
                thickness=2, lineType=cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
        if jumpFrame == 0:
            key = cv2.waitKey(0)
            
            while key not in [ord('q'), ord('k'), ord('p'), ord('c')]:
                key = cv2.waitKey(0)

            if cv2.waitKey(0) & 0xFF == ord('c'):
                jumpFrame = 21
     
        

    cap.release()
    cv2.destroyAllWindows()


# print(frames)

20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0
0
0
0
0
0
0
0
0
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0
0
0
0
0
0
0
0
0
0
20
19